<center>  <h1> Lab 3: Batch Normalization </h1> </center> 
<center> Krishna Pillutla, Zaid Harchaoui </center>
    <center> Data 598 (Winter 2022), University of Washington </center>

In this lab, we will study batch normalization. 
For simplicity, we will stick to multilayer perceptrons (MLPs) 
but the same ideas extend directly to convolutional neural networks.


## Convolution review
Before we start, we quickly look at convolutions in practice. We will look at the input size, input channels, output channels, etc.

The arguments are:

- in_channels: number of channels in the input image
- out_channels: number of channels in the output image. Equivalently, the number of filters
- kernel size: the size of kernel/filter. E.g. $3 \times 3$ or $5 \times 5$
- padding: how much zero padding to perform. Everything beyond the padding is dropped
- stride: the stride of the convolution

In [16]:
import torch
convolution_layer = torch.nn.Conv2d(
    in_channels=3, out_channels=16, kernel_size=5,
    padding=3, stride=3
)
# this is an instance of `torch.nn.Module`
print(isinstance(convolution_layer, torch.nn.Module))

True


In [17]:
image_size = 224
random_images = torch.randn(1, 3, image_size, image_size) 
# 1 is the batch dimension; 3 is the number of channles
# batch dimension: the convolution operation is applied in parallel for element of the batch

output = convolution_layer(random_images)
## Exercise: parse the shape of the output
print(output.shape)

torch.Size([1, 16, 76, 76])


In [3]:
image_size = 574
random_images = torch.randn(8, 3, image_size, image_size)  
# 8 is the batch dimension; 3 is the number of channels

output = convolution_layer(random_images)
## Exercise: parse the shape of the output
print(output.shape)

torch.Size([8, 16, 570, 570])


Note: the convolution layer only requires the number of input and output channels. 
The size of the image or the size of the batch does not matter! Why do you think this is the case?

Ans:
This is because the kernel will be applied to whole image size (and rest will be padded) and batch does not matter since it has to be applied to all the images

**Exercises**: 
- Play with the image sizes above to understand the outputs you get
- Change the padding and the stride. See if you can figure out a formula between how the output size relates to the input and kernel sizes.

In [21]:
parameters = dict(convolution_layer.named_parameters())
print('parameter names:', parameters.keys())

print(parameters['weight'].shape)  # what does the shape denote?

#torch.Size([16, 3, 5, 5])
#5 - kernel size
#3 - number of input channels
#16 - number of output channels

parameter names: dict_keys(['weight', 'bias'])
torch.Size([16, 3, 5, 5])


## Standardization in linear regression: An optimization viewpoint

Recall that we always normalize our features to be zero mean and unit variance 
(or something similar) on our training set.

Let us stack our input data $x_1, \cdots, x_n \in \mathbb{R}^d$ into a matrix $X \in \mathbb{R}^{n \times d}$. Denote the coordinate-wise mean and standard deviation as 
$$
    \mu^{(j)} = \frac{1}{n} \sum_{i=1}^n x_i^{(j)} \,, \quad
    \sigma^{(j)} = \sqrt{\frac{1}{n} (x_i^{(j)} - \mu^{(j)})^2} \,,
$$
for $j = 1, \cdots, d$. 
We can simply write the standardization as
$$
    \hat x_i = \frac{1}{\sigma}(x_i - \mu)
$$
where the operations are applied elementwise. 
We have applied this transformation to our data in each lab so far. 


Consider the linear prediction model 
$
f(x) = w^\top x + b
$
to predict outputs $y \approx f(x)$ from inputs $x$. Given $n$ input-output samples $(x_1, y_1), \ldots, (x_n, y_n) \in \mathbb{R}^d \times \mathbb{R}$, minimizing the least square loss over the training data reads
$$
\min_{w,b} \frac{1}{n}\sum_{i=1}^n  (y_i - w^\top x_i - b)^2.
$$

**Mean Normalization (Homework exercise)**:
Show that minimizing in $b$ amounts to center both inputs and outputs leading to 
$$
\min_{w} f(w) = \frac{1}{n}\sum_{i=1}^n  (\widetilde y_i - w^\top \hat x_i)^2.
$$
where $\widetilde y$ and $\hat x$ are respectively the centered outputs and inputs.


Recall now that from an optimization point of view, the step-size of a gradient step is controlled by the smoothness of the function, i.e. the Lipschitz-continuity of its gradient, which (for twice continuously differentiable functions) can simply computed as the largest eigenvalue of the Hessian matrix. 

**Variance Normalization**:
Similar to above, one can show that 
after normalizing the inputs, the objective  $f(w) = \frac{1}{n}\sum_{i=1}^n  (\widetilde y_i - w^\top \widehat x_i)^2$ has a smoothness constant no bigger than d.

In other words, feature normalization allows one to use a standard learning rate, invariant to the scale of the input. 

## Batch Normalization

We standardize our inputs in neural networks as well, but that only ensures that inputs to the first layer are normalized. What about inputs to subsequent layers?

**What is batch norm?**

Batch normalization is a technique which generalizes this standardization operation to ensure normalization of inputs in to an intermediate layer. 

In the linear model case, the mean and variance of our features do not change (since the data is fixed), therefore precomputing them once suffices. 
However, the inputs to hidden layers in neural networks changes over time. 
Batch norm, therefore, computes the mean and variance over a *minibatch of data*.

The batch norm layer gives the network two *trainable* parameters $\beta, \gamma$
which are learned over the course of training. If $\hat x_i$ is the normalized 
version of input $x_i$ to a batch norm layer, the output of the batch norm layer 
is 
$\gamma \hat x_i + \beta$.

**Note**: Batch norm requires large minibatch sizes such as 64 or 128 to work.

The pseudocode is given below. 

<img src="https://melfm.github.io/posts/2018-08-Understanding-Normalization/img/batch_norm_forward.png" width="500">


### Batch norm in action
Our first step is to notice that batch norm behaves differently at train and test times. 

At test time, there is no reason for a model to receive a large enough minibatch of inputs (think about the deep networks running on your smart phone).
Batch norm does not work if the minibatch size is too small. 

A solution to this issue is to estimate a running mean and standard deviation of the data encountered during training and use this is a proxy at test time. 

As a result, we need to tell PyTorch to switch from training to evaluation mode and vice-versa using `model.train()` and `model.eval()`, where `model` is a `torch.nn.Module`. 

## Part 1: Batch norm in action
Let us visualize how batch norm behaves differently at train and test times on a multilayer perceptron (MLP).


In [22]:
import numpy as np
import torch
from torchvision.datasets import FashionMNIST
from torch.nn.functional import cross_entropy, relu
import time
import copy

import matplotlib.pyplot as plt 
%matplotlib inline 

torch.manual_seed(0)
np.random.seed(1)

We use PyTorch's `BatchNorm1d` function. It is called so because the 
input a batch of vectors in $\mathbb{R}^d$ (in the form of a matrix of size $n\times d$). 

In [23]:
# pass in input dimensionality (it needs to create mean/std vectors of this dimension)
batch_norm_layer = torch.nn.BatchNorm1d(num_features=10)

# This is an instance of torch.nn.Module; we can use it as we would a module
isinstance(batch_norm_layer, torch.nn.Module)

True

In [30]:
# Let us create some synthetic data with a nonzero mean
mean_vector = 20 * torch.rand(10)  # non-zero mean
std_vector = 10 * torch.rand(10)

def get_random_data():
    return std_vector[None] * torch.randn(25, 10) + mean_vector[None]  # mean = mean_vector
random_data = get_random_data()

## Let us go to evaluation mode
batch_norm_layer.eval()

# print mean and variance of the data
print('Input mean:')
print(random_data.mean(dim=0))

print('Input std:')
print(random_data.std(dim=0))

Input mean:
tensor([ 1.2290,  8.2977, 17.4594,  6.0496,  5.0881,  4.7807,  9.4291,  1.7779,
         1.1503, 19.8451])
Input std:
tensor([2.5954, 3.4691, 5.9597, 0.4528, 3.8345, 5.8139, 3.2988, 4.0678, 5.6456,
        0.0209])


In [31]:
# Let us run our data through the batch norm layer and register its output
output = batch_norm_layer(random_data) # <TODO: your code here> apply `batch_norm_layer` to `random_data`

print('Output mean:')
print(output.mean(dim=0))

print('Output std:')
print(output.std(dim=0))

Output mean:
tensor([ 1.2290,  8.2976, 17.4593,  6.0496,  5.0881,  4.7807,  9.4291,  1.7779,
         1.1503, 19.8450], grad_fn=<MeanBackward1>)
Output std:
tensor([2.5954, 3.4691, 5.9596, 0.4528, 3.8345, 5.8139, 3.2988, 4.0677, 5.6455,
        0.0209], grad_fn=<StdBackward0>)


We do not observe any difference because batch norm in evaluation mode uses 
historical statistics of mean and std. But since this batch norm layer was just initialized, 
it has no history to go by.

Hence, it uses its defaults: $\mu = 0$ and $\sigma = 1$, which makes this batch norm layer just the identity map.

Let us switch to the train mode now. 


In [32]:
batch_norm_layer.train()  # switch to train mode

output = batch_norm_layer(random_data)# <TODO: your code here> apply `batch_norm_layer` to `random_data`

print('Output mean:')
print(output.mean(dim=0))

print('Output std:')
print(output.std(dim=0))

Output mean:
tensor([ 2.8610e-08,  1.2398e-07,  4.2915e-08, -4.4823e-07, -2.8610e-08,
         0.0000e+00, -7.8678e-08,  1.1921e-08, -1.4305e-08,  4.3726e-05],
       grad_fn=<MeanBackward1>)
Output std:
tensor([1.0206, 1.0206, 1.0206, 1.0206, 1.0206, 1.0206, 1.0206, 1.0206, 1.0206,
        1.0086], grad_fn=<StdBackward0>)


And voilà! It works as expected. Let us run a few more random samples and investigate the evaluation mode further. 

In [33]:
for i in range(1000):  # so that batch norm updates its internal statistics
    random_data = get_random_data()
    output = batch_norm_layer(random_data)
    
batch_norm_layer.eval()  # switch to eval mode

random_data = get_random_data()
# print mean and variance of the data
print('Input mean:')
print(random_data.mean(dim=0))

print('Input std:')
print(random_data.std(dim=0))

output = batch_norm_layer(random_data)

print('Output mean:')
print(output.mean(dim=0))

print('Output std:')
print(output.std(dim=0))

Input mean:
tensor([ 2.4337, 11.4200, 17.0795,  6.0232,  2.9947,  5.7541,  9.0437,  1.9582,
         1.9884, 19.8391])
Input std:
tensor([3.0015, 3.7518, 3.2199, 0.5087, 4.4398, 4.9793, 3.1715, 3.4411, 4.6568,
        0.0298])
Output mean:
tensor([ 0.1999,  0.2792,  0.0688,  0.1627, -0.4313,  0.0950, -0.0233,  0.2644,
         0.1290, -0.1525], grad_fn=<MeanBackward1>)
Output std:
tensor([1.1000, 0.9450, 0.8250, 0.8673, 1.0863, 0.7997, 1.0950, 0.7424, 0.9724,
        1.3195], grad_fn=<StdBackward0>)


Our data is much better normalized now!

## Part 2: Batch norm + MLPs
Just as in the convex case, batch norm helps with learning rates in the nonconvex case as well, especially when the networks are very deep. 

We start with a simple MLP module. Please read it carefully and understand what is going on. 

In [ ]:
class MultiLayerPerceptron(torch.nn.Module):
    def __init__(self, input_dim, output_dim, 
                 num_hidden_layers, hidden_width, 
                 use_batch_norm=False):
        ## pass in how many hidden layers to use and width of each layer
        ## We will use the same width for all hidden layers
        super().__init__()  # call constructor of a super class
        self.num_hidden_layers = num_hidden_layers
        
        # Construct linear layers: use ModuleList to hold a list of submodules
        self.hidden_layers = torch.nn.ModuleList([torch.nn.Linear(input_dim, hidden_width)])
         # input -> hidden
        
        for i in range(num_hidden_layers-1):
            # hidden i -> hidden i+1
            self.hidden_layers.append(torch.nn.Linear(hidden_width, hidden_width))
            
        # hidden -> output
        self.hidden_to_output = torch.nn.Linear(hidden_width, output_dim)
            
        # construct batch norm layers
        self.use_batch_norm = use_batch_norm
        if use_batch_norm:
            print('using batch norm')
            self.batch_norm_layers = torch.nn.ModuleList(
                [torch.nn.BatchNorm1d(hidden_width)
                 for _ in range(num_hidden_layers)]
            )
        else:
            print('not using batch norm')
            
    def forward(self, x,):
        for i in range(self.num_hidden_layers):
            # apply self.hidden_layers[i] to x
            x = # <TODO: your code here> 
            # apply the `relu` function to x
            x = # <TODO: your code here> 
            # apply batch norm if required
            if self.use_batch_norm:
                # apply `batch_norm_layers[i]` to x
                x = # <TODO: your code here> 
        # hidden -> output
        # apply `hidden_to_output` to x
        x = # <TODO: your code here> 
        return x
            

In [ ]:
model = MultiLayerPerceptron(input_dim=784, output_dim=10, hidden_width=64,  
                              num_hidden_layers=3, 
                              use_batch_norm=True)

print(model) # Do you see all the weights and 

Let us load our FashionMNIST dataset again.

In [ ]:
# download dataset (~117M in size)
train_dataset = FashionMNIST('./data', train=True, download=True)
X_train = train_dataset.data # torch tensor of type uint8
y_train = train_dataset.targets # torch tensor of type Long
test_dataset = FashionMNIST('./data', train=False, download=True)
X_test = test_dataset.data
y_test = test_dataset.targets

# choose a subsample of 10% of the data:
idxs_train = torch.from_numpy(
    np.random.choice(X_train.shape[0], replace=False, size=X_train.shape[0]//10))
X_train, y_train = X_train[idxs_train], y_train[idxs_train]
# idxs_test = torch.from_numpy(
#     np.random.choice(X_test.shape[0], replace=False, size=X_test.shape[0]//10))
# X_test, y_test = X_test[idxs_test], y_test[idxs_test]

print(f'X_train.shape = {X_train.shape}')
print(f'n_train: {X_train.shape[0]}, n_test: {X_test.shape[0]}')
print(f'Image size: {X_train.shape[1:]}')

# Normalize dataset: pixel values lie between 0 and 255
# Normalize them so the pixelwise mean is zero and standard deviation is 1

X_train = X_train.float()  # convert to float32
X_train = X_train.view(-1, 784)  # flatten into a (n, d) shape
mean, std = X_train.mean(axis=0), X_train.std(axis=0)
X_train = (X_train - mean[None, :]) / (std[None, :] + 1e-6)  # avoid divide by zero

X_test = X_test.float()
X_test = X_test.view(-1, 784)
X_test = (X_test - mean[None, :]) / (std[None, :] + 1e-6)

n_class = np.unique(y_train).shape[0]

We now write the training utilities. 

**Note**: We must pay extra care to ensure that the model is in training or evaluation mode when appropriate.

In [ ]:
def compute_objective(model, X, y):
    """ Compute the multinomial logistic loss. 
        model is a module
        X of shape (n, d) and y of shape (n,)
    """
    # TODO: your code here
    # compute the `cross_entropy` loss between `model` applied to `X` and `y`

@torch.no_grad()
def compute_accuracy(model, X, y):
    """ Compute the classification accuracy
        ws is a list of tensors of consistent shapes 
        X of shape (n, d) and y of shape (n,)
    """
    is_train = model.training  # if True, model is in training mode
    model.eval()  # use eval mode for accuracy
    score = model(X)
    predictions = torch.argmax(score, axis=1)  # class with highest score is predicted
    if is_train:  # switch back to train mode if appropriate
        model.train()
    return (predictions == y).sum() * 1.0 / y.shape[0]

@torch.no_grad()
def compute_logs(model, verbose=False):
    is_train = model.training  # if True, model is in training mode
    model.eval()  # switch to eval mode
    train_loss = compute_objective(model, X_train, y_train)
    test_loss = compute_objective(model, X_test, y_test)
    train_accuracy = compute_accuracy(model, X_train, y_train)
    test_accuracy = compute_accuracy(model, X_test, y_test)
    if verbose:
        print(('Train Loss = {:.3f}, Train Accuracy = {:.3f}, ' + 
               'Test Loss = {:.3f}, Test Accuracy = {:.3f}').format(
                train_loss.item(), train_accuracy.item(), 
                test_loss.item(), test_accuracy.item())
    )
    if is_train:  # switch back to train mode if appropriate
        model.train()
    return (train_loss, train_accuracy, test_loss, test_accuracy)

Now we write the minibatch SGD function (copied over from a previous lab). 
Note again that we must pay special care to making sure the model is in training mode

In [ ]:
def minibatch_sgd_one_pass(model, X, y, learning_rate, batch_size, verbose=False):
    model.train()
    num_examples = X.shape[0]
    average_loss = 0.0
    num_updates = int(round(num_examples / batch_size))
    for i in range(num_updates):
        idxs = np.random.choice(X.shape[0], size=(batch_size,)) # draw `batch_size` many samples
        model.train()  # make sure we are in train mode
        # compute the objective. 
        objective = # <TODO: your code here> 
        
        average_loss = 0.99 * average_loss + 0.01 * objective.item()
        if verbose and (i+1) % 100 == 0:
            print(average_loss)
        
        # compute the gradient and make SGD update with learning rate
        # <TODO: your code here> 
        
    return model

Our goal now is to find the divergent learning rates 
with and without batch norm. We use a fixed batch size of 32

Use a hidden width of 64 throughout. 
Vary the depth of the network
(i.e., the number of hidden layers)
between 1 and 8.

In [ ]:
num_hidden_layers = 1

# no batch norm
model = MultiLayerPerceptron(input_dim=784, output_dim=10, hidden_width=64,  
                              num_hidden_layers=num_hidden_layers, 
                              use_batch_norm=False)


learning_rate = # <TODO: your values here> 

logs = []

logs.append(compute_logs(model, verbose=True))

batch_size = 32

for _ in range(batch_size*2):  # run multiple passes because our sub-sampled dataset is too small
    model = minibatch_sgd_one_pass(model, X_train, y_train, learning_rate, 
                                   batch_size=batch_size, verbose=True)


In [ ]:
# with batch norm
model = MultiLayerPerceptron(input_dim=784, output_dim=10, hidden_width=64,  
                              num_hidden_layers=num_hidden_layers, 
                              use_batch_norm=True)


learning_rate = # <TODO: your values here> 

logs = []

logs.append(compute_logs(model, verbose=True))

batch_size = 32

for _ in range(batch_size):  # run multiple passes because our sub-sampled dataset is too small
    model = minibatch_sgd_one_pass(model, X_train, y_train, learning_rate, 
                                   batch_size=batch_size, verbose=True)


Redo the last part with depths of $10$ and $25$, and $50$. What do you observe?

**NOTE**: the larger models (depth > 20) get very slow. Use a cloud computing resource if it is too slow on your laptop.

**Hint**: If some of the models at these depths get stuck or suddenly diverge to infinity/NaN, know that it is an often encountered problem with very deep networks. A combination of many factors can overcome this issue: careful initialization and tuning of learning rates as well as dropout. The most popular fix, however, is to use residual networks, know as _ResNets_. We will talk about these in class.